In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction,features_saving,features_loading)

from utils.utils import (create_submission ,metrics2csv, save_results)

# Checking for resources avaliables

In [2]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         91G         49G        8.4G         79G        118G
Swap:            0B          0B          0B
Sun Nov 26 18:54:32 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   64C    P0    79W / 149W |  10418MiB / 11439MiB |     91%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [3]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [4]:
use_gpu = True
use_DataParalel= True 
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS



# Path to features

In [5]:
path2features= "/mnt/home/e209440/state-farm-distracted-driver-detection/features/" 
#path2features= "/mnt/home/r120084/state-farm-distracted-driver-detection/features/" 

# Choosing Model

In [ ]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

In [ ]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

# Loading features

In [ ]:
model_name

In [ ]:
load_feat= features_loading(path2features,model_name)

In [ ]:
load_feat['train'][0].shape,load_feat['train'][1].shape

In [ ]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [ ]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [ ]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

# Trainning Model

In [ ]:
path2saveModel = '/mnt/home/r120084/project/models/'+model_name+'distractdriver'
#path2saveModel = '/mnt/home/e209440/models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50

optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [ ]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [ ]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

# Loading best epoch

In [ ]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [ ]:
metrics2csv(trainer, model_name)

In [ ]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [ ]:
train_eval, valid_eval

# Dataset without shuffling 

#### Utilized to plot mismatch cases

In [ ]:
path2train = "/mnt/home/e209440/data/train"  

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [ ]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [ ]:
dsets = {
    'valid': KaggleSafeDriverDataset(path2train,
                                     transforms=data_transforms['valid'],
                                     use_only=use_only, is_val=True, val_size=0.2),
}

In [ ]:

dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size,
                                           pin_memory=False, use_shuffle= False)

# Evaluating resultings 

In [ ]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)

result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)

result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

In [ ]:
predictions_out = {'train': result_train, 'valid': result_valid, 'test': result_test}

In [ ]:
save_results(predictions_out, model_name)

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', len(dsets['train']) )
print('Valid: ', correct_valid, '/', len(dsets['valid']) )

# Make submission of the Test set

In [ ]:
create_submission(result_test, model_name + '_distracted_driver')